In [1]:
!pip install pyarrow
!pip install wrds


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import sys
import datetime as dt
import os
import pickle
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import numpy as np
import pandas as pd
import os
import sys
import pyarrow.parquet as pq
import re
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import wrds
import pytz
import math
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize
from tqdm import tqdm

In [3]:
def get_sd_and_ed_for_returns(date, lead):
    start_current_month = date.replace(day=1)
    start_current_month = start_current_month + relativedelta(months = lead)
    start_next_month = start_current_month + relativedelta(months=1)
    #start_current_month = start_current_month + relativedelta(months=-2)

    start_current_month_str = start_current_month.strftime("%Y-%m-%d")
    start_next_month_str = start_next_month.strftime("%Y-%m-%d")

    return start_current_month_str, start_next_month_str

In [4]:
### log-in using your credentials
if 'conn' in locals():
    print("WRDS connection already open!")
else:
    conn = wrds.Connection()

Enter your WRDS username [datalore]: anujchoudhury
Enter your password: ············
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?:  y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [5]:
def get_req_tick_cik_map(start_date, end_date):
    # Assuming you have already imported the necessary libraries and connected to your database
    # Assuming you have already imported the necessary libraries and connected to your database

    print(start_date)
    print(end_date)

    sql_query = f"""
    SELECT distinct cik, tic, cusip, cshtrm, trt1m, cmth
    FROM comp_na_daily_all.secm
    WHERE datadate >= '{start_date}' and datadate < '{end_date}'
    """

    cik_tick_map = conn.raw_sql(sql_query)
    cik_tick_map = cik_tick_map.dropna(subset=['cik', 'trt1m'])
    cik_tick_map['cik'] = cik_tick_map['cik'].astype(int)

    print("Total rows:", len(cik_tick_map))
    print("Unique cusip count:", cik_tick_map['cusip'].nunique())

    # Identify 'cik' values with multiple rows
    cik_counts = cik_tick_map['cik'].value_counts()
    multiple_cik_values = cik_counts[cik_counts > 1].index.tolist()

    # Create a mask to filter rows for 'cik' values with multiple rows
    multiple_cik_mask = cik_tick_map['cik'].isin(multiple_cik_values)

    # Create a mask to identify rows with null 'cshtrm'
    null_cshtrm_mask = cik_tick_map['cshtrm'].isnull()

    # Apply both masks to filter rows for 'cik' values with multiple rows
    filtered_cik_tick_map = cik_tick_map[multiple_cik_mask & ~null_cshtrm_mask]

    # Group by 'cik' and apply idxmax() to find the index with the largest 'cshtrm'
    grouped_map = filtered_cik_tick_map.groupby(['cik'])['cshtrm'].idxmax()

    # Use loc to select the rows with the largest 'cshtrm' for 'cik' values with multiple rows
    cik_tick_map_with_max_cshtrm = filtered_cik_tick_map.loc[grouped_map]


    # Select rows for 'cik' values that have only one row
    single_row_cik_values = cik_counts[cik_counts == 1].index.tolist()
    single_row_cik_mask = cik_tick_map['cik'].isin(single_row_cik_values)
    cik_tick_map_single_rows = cik_tick_map[single_row_cik_mask]

    # Append rows for 'cik' values with one row to the previously selected rows
    final_cik_tick_map = pd.concat([cik_tick_map_with_max_cshtrm, cik_tick_map_single_rows], axis = 0)
    # Rename the 'trt1m' column to 'ret'
    final_cik_tick_map.rename(columns={'trt1m': 'ret'}, inplace=True)

    print("Total rows after selecting rows with max cshtrm:", len(final_cik_tick_map))


    return final_cik_tick_map

In [6]:
def get_n_mnths_return(date_obj, lead):
    sd,ed = get_sd_and_ed_for_returns(date_obj, lead)
    cik_tick_map = get_req_tick_cik_map(start_date=sd, end_date=ed)
    
    return cik_tick_map
    

In [7]:
date_obj = datetime.strptime('Dec1996', '%b%Y')
data = get_n_mnths_return(date_obj, 2)

1997-02-01
1997-03-01
Total rows: 11093
Unique cusip count: 10873
Total rows after selecting rows with max cshtrm: 10670


In [8]:
month_dict = { 1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
folder_path = r'/data/workspace_files/parquet_files'

In [9]:
def create_directories():
    folder_path = '/data/workspace_files/parquet_files'
    n_bucket = 5
    for lead in range(1,13):
        for sim_type in ['cosine_similarity', 'jaccard_similarity']:
            for quartile in range(1,n_bucket + 1):
                folder_path_req = f'{folder_path}/portfolio_lead_lag/{sim_type}/{lead}/Q{quartile}'
                print(folder_path_req)
                os.makedirs(folder_path_req, exist_ok = True)
            folder_path_req = f'{folder_path}/portfolio_lead_lag/{sim_type}/{lead}/LS'
            print(folder_path_req)
            os.makedirs(folder_path_req, exist_ok = True)

In [10]:
create_directories()

/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/1/Q1
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/1/Q2
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/1/Q3
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/1/Q4
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/1/Q5
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/1/LS
/data/workspace_files/parquet_files/portfolio_lead_lag/jaccard_similarity/1/Q1
/data/workspace_files/parquet_files/portfolio_lead_lag/jaccard_similarity/1/Q2
/data/workspace_files/parquet_files/portfolio_lead_lag/jaccard_similarity/1/Q3
/data/workspace_files/parquet_files/portfolio_lead_lag/jaccard_similarity/1/Q4
/data/workspace_files/parquet_files/portfolio_lead_lag/jaccard_similarity/1/Q5
/data/workspace_files/parquet_files/portfolio_lead_lag/jaccard_similarity/1/LS
/data/workspace_files/parquet_files/portfolio_lead_lag/cos

In [11]:
folder_path = '/data/workspace_files/parquet_files'
for score_type in ['cosine_similarity', 'jaccard_similarity']:
    for lead in range(1,13):  
        for year in range(1996, 2018):
            print(f"Running score_type {score_type}, lead :{lead}, year : {year}")
            for month in tqdm(range(1, 13)):
                if year == 1996 and month <= 1:
                    continue
                month_yr = f"{month_dict[month]}{year}"
                date_obj = datetime.strptime(month_yr, '%b%Y')
                check_write_file = f'{folder_path}/portfolio_lead_lag/{score_type}/{lead}/Q{5}/{month_yr}.parquet'
                if os.path.exists(check_write_file):
                    print(f"{check_write_file} exists ")
                    continue
                cik_tick_map_req = get_n_mnths_return(date_obj, lead)
                score_path = f'{folder_path}/similarity_scores/{month_yr}.parquet'
                scores = pq.read_table(score_path).to_pandas()
                cik_tick_map_req['ret_clipped'] = cik_tick_map_req['ret'].clip(lower=-50.0, upper=50.0)
                scores = pd.merge(scores, cik_tick_map_req, how='inner', on=['cik'])
                for num_buckets in [5]:
                    # Create quantile buckets
                    buckets = pd.qcut(scores[score_type], q=num_buckets, labels=False)
                    if month == 1:
                        prev_month = 12
                        prev_yr = year-1
                    else:
                        prev_month = month-1
                        prev_yr = year
                    prev_month_yr = f"{month_dict[prev_month]}{prev_yr}"
                    for i in range(num_buckets):
                        for history in [6]:
                            print(f'Running for lookback window = {history} and leading period = {lead}')
                            # Filter scores for the current quantile
                            quantile_positions = scores[buckets == i].copy(deep = True)
                            quantile_positions['pos_type'] = 'Quantile_' + str(i+1)
                            quantile_positions['num_months'] = 1

                            port_path = f'{folder_path}/portfolio_lead_lag/{score_type}/{lead}/Q{i+1}/{prev_month_yr}.parquet'
                            if os.path.exists(port_path):
                                prev_positions = pq.read_table(port_path).to_pandas().copy(deep = True)
                                prev_positions['num_months'] = prev_positions['num_months'] + 1
                                prev_positions = prev_positions[prev_positions['num_months'] <= history]
                                print(f'Prev Holdings {len(prev_positions)}')
                                quantile_positions = pd.concat([quantile_positions, prev_positions], axis = 0).reset_index(drop=True)

                            print(f'Before dropping dupes: {len(quantile_positions)}')
                            idxes = quantile_positions.groupby('cik')['num_months'].idxmin()
                            quantile_positions = quantile_positions.loc[idxes]

                            quantile_positions['weight'] = 1/len(quantile_positions)
                            quantile_positions['date'] = month_yr

                            print(f'After dropping dupes: {len(quantile_positions)}')
                            quantile_positions = quantile_positions[['date', 'cik', 'pos_type', 'num_months', 'weight', 'ret', 'ret_clipped', 'filing_date', 'cshtrm']]

                            # Save to Parquet
                            quantile_file_path = f'{folder_path}/portfolio_lead_lag/{score_type}/{lead}/Q{i+1}/{month_yr}.parquet'
                            quantile_positions.to_parquet(quantile_file_path)

                            print(f'Quantile {i+1} Holdings: {len(quantile_positions)}')


Total rows after selecting rows with max cshtrm: 11774
Running for lookback window = 6 and leading period = 8
Prev Holdings 1110
Before dropping dupes: 1868
After dropping dupes: 1576
Quantile 1 Holdings: 1576
Running for lookback window = 6 and leading period = 8
Prev Holdings 1162
Before dropping dupes: 1919
After dropping dupes: 1718
Quantile 2 Holdings: 1718
Running for lookback window = 6 and leading period = 8
Prev Holdings 1169
Before dropping dupes: 1926
After dropping dupes: 1731
Quantile 3 Holdings: 1731
Running for lookback window = 6 and leading period = 8
Prev Holdings 1173
Before dropping dupes: 1930
After dropping dupes: 1731
Quantile 4 Holdings: 1731
Running for lookback window = 6 and leading period = 8
Prev Holdings 1131
Before dropping dupes: 1889
After dropping dupes: 1508
Quantile 5 Holdings: 1508
2014-02-01
2014-03-01
Total rows: 13095
Unique cusip count: 11864
Total rows after selecting rows with max cshtrm: 11653
Running for lookback window = 6 and leading perio

100%|██████████| 12/12 [00:13<00:00,  1.12s/it]


In [12]:
def save_ls_portfolio(score_type, n_bucket):
    for year in range(1996,2018):
        for month in range(1, 13):
            for lead in range(1,13):
                if year == 1996 and month <= 1:
                        continue
                month_yr = f"{month_dict[month]}{year}"
                long_file_path = f'{folder_path}/portfolio_lead_lag/{score_type}/{lead}/Q{n_bucket}/{month_yr}.parquet'
                print(long_file_path)
                short_file_path = f'{folder_path}/portfolio_lead_lag/{score_type}/{lead}/Q1/{month_yr}.parquet'
                print(short_file_path)
                long_positions = pq.read_table(long_file_path).to_pandas()
                long_positions['pos_type'] = 1
                short_positions = pq.read_table(short_file_path).to_pandas()
                short_positions['pos_type'] = -1
                short_positions['weight'] = -1*short_positions['weight']
                positions = pd.concat([long_positions, short_positions])
                ls_file_path = f'{folder_path}/portfolio_lead_lag/{score_type}/{lead}/LS/{month_yr}.parquet'
                print(ls_file_path)
                positions.to_parquet(ls_file_path)

In [13]:
for score_type in ['cosine_similarity', 'jaccard_similarity']:
    for n_bucket in [5]:
        save_ls_portfolio(score_type, n_bucket)

/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/2/Q5/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/2/Q1/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/2/LS/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/3/Q5/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/3/Q1/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/3/LS/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/4/Q5/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/4/Q1/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/4/LS/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosine_similarity/5/Q5/Aug2015.parquet
/data/workspace_files/parquet_files/portfolio_lead_lag/cosin